# Лабораторная работа №6

## Подготовка входных данных

In [3]:
!pip install catboost

Импортируем необходимые библиотеки.

In [5]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [7]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [9]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [11]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [13]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [15]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем класс CatBoostClassifier для классификации датасета методами CatBoost.

In [21]:
from catboost import CatBoostClassifier

Создаём объект класса CatBoostClassifier, реализующий метод решающего дерева.

In [23]:
cat_boost_classifier = CatBoostClassifier()

Определяем параметры, которые мы хотим оптимизировать для CatBoostClassifier.
* “iterations”: этот ключ определяет количество итераций, которые CatBoost делает при обучении модели.
* ‘learning_rate’: этот ключ определяет скорость обучения модели CatBoost.
* ‘depth’: этот ключ определяет максимальную глубину деревьев в модели CatBoost.

In [25]:
cat_boost_params ={
    "iterations": range(1,6),
    'learning_rate': [0.1,0.25,0.5,0.75,1],
    'depth':range(1,6),
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [27]:
cat_boost_grid = GridSearchCV(cat_boost_classifier, cat_boost_params, cv=2, n_jobs=1)
cat_boost_grid.fit(X_train, y_train)

0:	learn: 0.6509462	total: 146ms	remaining: 0us
0:	learn: 0.6510369	total: 3.08ms	remaining: 0us
0:	learn: 0.5959840	total: 1.35ms	remaining: 0us
0:	learn: 0.5961903	total: 6.22ms	remaining: 0us
0:	learn: 0.5255874	total: 11.8ms	remaining: 0us
0:	learn: 0.5259705	total: 6.44ms	remaining: 0us
0:	learn: 0.4793168	total: 6.57ms	remaining: 0us
0:	learn: 0.4799379	total: 1.57ms	remaining: 0us
0:	learn: 0.4537951	total: 3.54ms	remaining: 0us
0:	learn: 0.4547944	total: 7.5ms	remaining: 0us
0:	learn: 0.6509462	total: 5.86ms	remaining: 5.86ms
1:	learn: 0.6162677	total: 16.2ms	remaining: 0us
0:	learn: 0.6510369	total: 2.05ms	remaining: 2.05ms
1:	learn: 0.6167114	total: 3.88ms	remaining: 0us
0:	learn: 0.5959840	total: 6.37ms	remaining: 6.37ms
1:	learn: 0.5363278	total: 15.9ms	remaining: 0us
0:	learn: 0.5961903	total: 1.48ms	remaining: 1.48ms
1:	learn: 0.5373067	total: 7.62ms	remaining: 0us
0:	learn: 0.5255874	total: 1.69ms	remaining: 1.69ms
1:	learn: 0.4661989	total: 3.04ms	remaining: 0us
0:	lear

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000020A7A5B6EA0>,
             n_jobs=1,
             param_grid={'depth': range(1, 6), 'iterations': range(1, 6),
                         'learning_rate': [0.1, 0.25, 0.5, 0.75, 1]})

Создаём объект класса CatBoostClassifier, с лучшими найденными параметрами.

In [29]:
best_cat_boost_model = CatBoostClassifier(**cat_boost_grid.best_params_)

Обучаем модель на тренировочных данных.

In [31]:
best_cat_boost_model.fit(X_train, y_train)

0:	learn: 0.4991118	total: 4.23ms	remaining: 16.9ms
1:	learn: 0.4252862	total: 7.92ms	remaining: 11.9ms
2:	learn: 0.3937544	total: 26.9ms	remaining: 17.9ms
3:	learn: 0.3793950	total: 33.5ms	remaining: 8.38ms
4:	learn: 0.3685763	total: 38.9ms	remaining: 0us


Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [33]:
cat_boost_predicted = best_cat_boost_model.predict(X_test)

Вывод отчёта.

In [35]:
print('Параметры:', cat_boost_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, cat_boost_predicted, digits=5))

Параметры: {'depth': 5, 'iterations': 5, 'learning_rate': 0.5}
Оценка:
               precision    recall  f1-score   support

       False    0.85877   0.95494   0.90430      9010
        True    0.75319   0.46684   0.57641      2654

    accuracy                        0.84388     11664
   macro avg    0.80598   0.71089   0.74036     11664
weighted avg    0.83475   0.84388   0.82970     11664

